Obtengo los datos de la API y los visualizo con PANDAS

In [ ]:
import requests

import pandas as pd
from datetime import date

import psycopg2
from dotenv import load_dotenv
import os 

from psycopg2.extras import execute_values

url = "https://api.coinlore.net/api/tickers/?start=0&limit=100"
response = requests.get(url)
datos= response.json()



df=pd.DataFrame.from_dict(datos['data']).reset_index().rename(columns={"index":'indice'})
df


Aplico un filtro de los datos que me interesan y les agrego la hora de Extaccion 

In [ ]:

dfFiltrado= df.loc[0:15,['indice','id','symbol','name','rank','price_usd','percent_change_24h','percent_change_1h','market_cap_usd']]

hora_actual = pd.to_datetime('now')
hora_formateada = hora_actual.strftime("%H:%M:%S")

dfFiltrado =dfFiltrado.assign(Hora=hora_formateada)



Cambio los tipos de datos

In [ ]:

#Lo convierto en float y despues los paso a int porque no me deja pasarlo directamente
dfFiltrado["price_usd"]= dfFiltrado["price_usd"].astype("float64")
#dfFiltrado["price_usd"]= dfFiltrado["price_usd"].astype("int")

dfFiltrado["percent_change_24h"]= dfFiltrado["percent_change_24h"].astype("float64")
#dfFiltrado["percent_change_24h"]= dfFiltrado["percent_change_24h"].astype("int")

dfFiltrado["percent_change_1h"]= dfFiltrado["percent_change_1h"].astype("float64")
#dfFiltrado["percent_change_1h"]= dfFiltrado["percent_change_1h"].astype("int")

dfFiltrado["market_cap_usd"]= dfFiltrado["market_cap_usd"].astype("float64")
#dfFiltrado["market_cap_usd"]= dfFiltrado["market_cap_usd"].astype("int")
#dfFiltrado["Year"]= dfFiltrado["Year"].astype("float64")

Conexion con Amazon redshift

In [6]:
#Prueba de conexion


load_dotenv()

url = os.getenv('url-sql')
data_base= os.getenv('data_base')
user=os.getenv('user')
pwd=os.getenv('pwd')

try:
    conn = psycopg2.connect(
        host=url,
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Conectado a Postgres")
    
except Exception as e:
    print("No es posible conectarse a Postgres")
    print(e)

Conectado a Postgres


Creacion de tabla en redshift

In [ ]:

def cargar_en_postgres(conn, table_name, dataframe):
    dtypes= dataframe.dtypes
    cols= list(dtypes.index )
    tipos= list(dtypes.values)
    type_map = {'int64': 'INT','int32': 'INT','float64': 'FLOAT','object': 'VARCHAR(50)','bool':'BOOLEAN'}
    sql_dtypes = [type_map[str(dtypes)] for dtypes in tipos]
    # Definir formato SQL VARIABLE TIPO_DATO
    column_defs = [f"{name} {data_type}" for name, data_type in zip(cols, sql_dtypes)]
    # Combine column definitions into the CREATE TABLE statement
    table_schema = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {', '.join(column_defs)}
        );
        """
    # Crear la tabla
    cur = conn.cursor()
    cur.execute(table_schema)
    # Generar los valores a insertar
    values = [tuple(x) for x in dataframe.to_numpy()]
    # Definir el INSERT
    insert_sql = f"INSERT INTO {table_name} ({', '.join(cols)}) VALUES %s"
    # Execute the transaction to insert the data
    cur.execute("BEGIN")
    execute_values(cur, insert_sql, values)
    cur.execute("COMMIT")
    print('Proceso terminado')

In [ ]:
cargar_en_postgres(conn=conn, table_name= "ValoresCryptos", dataframe=dfFiltrado)